# QuantBuffet Interactive Notebook

This notebook allows you to run Python code for financial analysis using the QuantConnect API.

## Setup

First, run the cell below to install required packages and set up the environment:

In [ ]:
# Install required packages
%pip install requests pandas numpy matplotlib plotly

# Import necessary libraries
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Setup plotting
%matplotlib inline
plt.style.use('ggplot')

print("✅ Environment setup complete")

In [ ]:
%%javascript
// Message listener for incoming code
window.addEventListener('message', function(event) {
    console.log('Message received:', event.data);
    
    // Check if this is a code injection message
    if (event.data && event.data.type === 'jupyter-load-code') {
        const codeSnippets = event.data.code;
        console.log('Received code snippets:', codeSnippets);
        
        if (Array.isArray(codeSnippets) && codeSnippets.length > 0) {
            // For each code snippet, create a new cell
            codeSnippets.forEach(function(code) {
                if (code && code.trim()) {
                    console.log('Creating cell with code:', code.substring(0, 50) + '...');
                    
                    // Create a new code cell at the end
                    const cell = Jupyter.notebook.insert_cell_at_bottom('code');
                    cell.set_text(code);
                }
            });
            
            // Save the notebook
            Jupyter.notebook.save_checkpoint();
            console.log('Added', codeSnippets.length, 'cells to notebook');
            
            // Show a notification
            if (IPython && IPython.notification_area) {
                IPython.notification_area.get_widget('notebook').info(
                    'Added ' + codeSnippets.length + ' code cells from the webpage', 3000
                );
            }
        }
    }
}, false);

// Send a message to the parent indicating we're ready
try {
    window.parent.postMessage({ type: 'jupyter-ready' }, '*');
    console.log('Sent ready message to parent');
} catch (e) {
    console.error('Failed to send ready message:', e);
}

console.log('Message listener installed');

## QuantConnect API

Set up the QuantConnect API helper class:

In [ ]:
class QuantConnectAPI:
    """QuantConnect API wrapper for data access and backtesting"""
    def __init__(self, user_id=None, access_token=None):
        self.base_url = "https://api.quantconnect.com/api/v2"
        self.user_id = user_id
        self.access_token = access_token
        self.connected = False
        
        if user_id and access_token:
            self.test_connection()
    
    def set_credentials(self, user_id, access_token):
        """Set API credentials"""
        self.user_id = user_id
        self.access_token = access_token
        return self.test_connection()
    
    def test_connection(self):
        """Test API connection"""
        endpoint = "/projects/list"
        response = self._make_request("GET", endpoint)
        
        if response and "projects" in response:
            print(f"✅ Connected to QuantConnect API")
            print(f"   Found {len(response['projects'])} projects")
            self.connected = True
            return True
        else:
            print("❌ Connection failed. Check credentials.")
            self.connected = False
            return False
    
    def _make_request(self, method, endpoint, data=None):
        """Make API request to QuantConnect"""
        url = f"{self.base_url}{endpoint}"
        params = {"userId": self.user_id}
        
        if data is None:
            data = {}
        data["token"] = self.access_token
        
        headers = {"Content-Type": "application/json"}
        
        try:
            if method.upper() == "GET":
                response = requests.get(url, params=params, json=data, headers=headers)
            else:
                response = requests.post(url, params=params, json=data, headers=headers)
                
            if response.status_code == 200:
                return response.json()
            else:
                print(f"❌ API Error: Status {response.status_code}")
                print(f"Response: {response.text}")
                return None
        except Exception as e:
            print(f"❌ Request Error: {str(e)}")
            return None
    
    def get_market_data(self, symbol, resolution="daily", start_date=None, end_date=None):
        """Get historical market data for a symbol"""
        if not self.connected:
            print("❌ Not connected. Call set_credentials() first.")
            return None
            
        # Set default dates if not provided
        if not end_date:
            end_date = datetime.now().strftime('%Y-%m-%d')
        if not start_date:
            # Default to 1 year ago
            start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')
        
        data = {
            "symbols": [symbol],
            "resolution": resolution,
            "start": start_date,
            "end": end_date
        }
        
        response = self._make_request("POST", "/data/read", data)
        
        if response and "data" in response and symbol in response["data"]:
            symbol_data = response["data"][symbol]
            
            if len(symbol_data) == 0:
                print(f"No data found for {symbol} in the specified date range")
                return pd.DataFrame()
            
            df = pd.DataFrame(symbol_data)
            df['time'] = pd.to_datetime(df['time'])
            df = df.set_index('time')
            
            print(f"✅ Retrieved {len(df)} records for {symbol}")
            return df
        else:
            print(f"❌ Failed to retrieve data for {symbol}")
            return None

# Create a QuantConnect API instance
qc = QuantConnectAPI()

print("QuantConnect API helper initialized")
print("To use it, set your credentials:")
print("qc.set_credentials('YOUR_USER_ID', 'YOUR_ACCESS_TOKEN')")

## Example Usage

Here's how to use the QuantConnect API to fetch and analyze data:

In [ ]:
# Set your QuantConnect credentials
# Replace with your actual credentials
# qc.set_credentials('YOUR_USER_ID', 'YOUR_ACCESS_TOKEN')

# Get Apple stock data for the past year
# apple_data = qc.get_market_data('AAPL')

# Plot the data
# if apple_data is not None and len(apple_data) > 0:
#     plt.figure(figsize=(12, 6))
#     plt.plot(apple_data.index, apple_data['close'])
#     plt.title('AAPL Stock Price')
#     plt.xlabel('Date')
#     plt.ylabel('Price ($)')
#     plt.grid(True)
#     plt.show()